In [11]:
import plotly.offline as py
import plotly.graph_objs as graph_objs
import pandas as pd
import geopandas as gpd
import numpy as np
import re

mapbox_access_token = "pk.eyJ1IjoibWlzaGtpY2UiLCJhIjoiY2s5MG94bWRoMDQxdjNmcHI1aWI1YnFkYyJ9.eFsHqEMYY7qxa0Pb9USCtQ"

In [27]:
dfg = gpd.read_file('data/NeighborhoodTabulationAreas.geojson')
dfg.columns = map(str.lower, dfg.columns)
dfg = dfg.applymap(lambda s:s.lower() if type(s) == str else s)

dfgB = gpd.read_file('data/BoroughBoundaries.geojson')
dfgB.columns = map(str.lower, dfgB.columns)
dfgB = dfgB.applymap(lambda s:s.lower() if type(s) == str else s)
dfgB.rename(columns={'boro_name': 'borough'}, inplace = True)

In [28]:
dfg.head()

,ntacode,shape_area,county_fips,ntaname,shape_leng,boro_name,boro_code,geometry
0,bk88,54005019.048,047,borough park,39247.2278309,brooklyn,3,"MULTIPOLYGON (((-73.97605 40.63128, -73.97717 ..."
1,qn51,52488277.846,081,murray hill,33266.9047967,queens,4,"MULTIPOLYGON (((-73.80379 40.77561, -73.80099 ..."
2,qn27,19726845.6282,081,east elmhurst,19816.7120736,queens,4,"MULTIPOLYGON (((-73.86110 40.76366, -73.85993 ..."
3,qn07,22887772.8652,081,hollis,20976.3355739,queens,4,"MULTIPOLYGON (((-73.75726 40.71814, -73.75589 ..."
4,mn06,10647077.5264,061,manhattanville,17040.6854129,manhattan,1,"MULTIPOLYGON (((-73.94608 40.82126, -73.94640 ..."


In [29]:
dfgB.head()

,boro_code,borough,shape_area,shape_leng,geometry
0,2,bronx,1186612478.27,462958.187578,"MULTIPOLYGON (((-73.89681 40.79581, -73.89694 ..."
1,5,staten island,1623757282.78,325956.009,"MULTIPOLYGON (((-74.05051 40.56642, -74.05047 ..."
2,3,brooklyn,1937593022.64,738745.842115,"MULTIPOLYGON (((-73.86706 40.58209, -73.86769 ..."
3,4,queens,3045878293.21,904188.424488,"MULTIPOLYGON (((-73.83668 40.59495, -73.83678 ..."
4,1,manhattan,636602658.887,361212.476098,"MULTIPOLYGON (((-74.01093 40.68449, -74.01193 ..."


In [30]:
full_df = pd.read_csv('data/Trees_Properties_With_Centroids.csv')
full_dfB = pd.read_csv('data/Trees_Properties_With_Centroids_Boro.csv')

In [31]:
full_df.head()

,Unnamed: 0,ntaname,borough,#trees,area,health,trees/sq.mile,#properties,avg.landprice_thous$/acre,avg.propvalue_thous$/acre,properties/sq.mile,centerLong,centerLat,hover
0,0,airport,queens,155.0,8.19,14.32,18.94,2.0,1995.03,4694.50,0.24,-73.745906,40.636482,airport<br>queens<br>trees/sq.mile: 18.94<br>a...
1,1,allerton-pelham gardens,bronx,3609.0,1.14,13.94,3174.61,5359.0,18.24,512.24,4713.98,-73.847425,40.864472,allerton-pelham gardens<br>bronx<br>trees/sq.m...
2,2,annadale-huguenot-prince's bay-eltingville,staten island,12538.0,5.06,13.42,2477.99,8389.0,19.11,619.14,1657.99,-74.187700,40.528667,annadale-huguenot-prince's bay-eltingville<br>...
3,3,arden heights,staten island,6729.0,1.81,13.91,3721.30,7702.0,9.66,448.19,4259.39,-74.188483,40.552659,arden heights<br>staten island<br>trees/sq.mil...
4,4,astoria,queens,4183.0,1.41,14.04,2964.63,8147.0,42.31,895.98,5774.05,-73.919697,40.761494,astoria<br>queens<br>trees/sq.mile: 2964.63<br...


Add bins column

In [32]:
bins = [
    "0-500",
    "501-1000",
    "1001-1500",
    "1501-2000",
    "2001-2500",
    "2501-3000",
    "3001-3500",
    "3501-4000",
    "4001-4500",
    "4501-5000"
]

binsB = [
    "2000-2300",
    "2301-2600",
    "2601-2900",
    "2901-3200",
    "3201-3500"
]

In [33]:
full_df['bin']=full_df['trees/sq.mile'].apply(lambda x: bins[0] if int(x) <=500 else bins[1] if int(x) <=1000 else bins[2] if x <=1500 else bins[3] if int(x) <=2000 else bins[4] if int(x) <=2500 else bins[5] if int(x) <=3000 else bins[6] if int(x) <=3500 else bins[7] if int(x) <=4000 else bins[8] if int(x) <=4500 else bins[9] )

In [38]:
full_dfB['bin']=full_dfB['trees/sq.mile'].apply(lambda x: binsB[0] if int(x) <=2300 else binsB[1] if int(x) <2600 else binsB[2] if x <=2900 else binsB[3] if int(x) <=3200 else binsB[4])

In [39]:
full_dfB.head()

,Unnamed: 0,borough,health,trees/sq.mile,avg.landprice_thous$/acre,properties/sq.mile,centerLong,centerLat,hover,bin
0,0,bronx,13.897568,2575.768919,56.952162,3636.815946,-73.897843,40.796350,bronx<br>trees/sq.mile: 2575.7689189189186<br>...,2301-2600
1,1,brooklyn,13.806863,2840.467843,37.254118,5551.347255,-73.866373,40.586820,brooklyn<br>trees/sq.mile: 2840.4678431372545<...,2601-2900
2,2,manhattan,13.460000,3307.277692,219.828846,8028.133846,-74.010173,40.685134,manhattan<br>trees/sq.mile: 3307.277692307693<...,3201-3500
3,3,queens,13.878929,2678.568036,35.663571,4278.410714,-73.837516,40.592832,queens<br>trees/sq.mile: 2678.568035714287<br>...,2601-2900
4,4,staten island,13.924444,2130.842778,24.254444,2752.206111,-74.050069,40.565911,staten island<br>trees/sq.mile: 2130.842777777...,2000-2300


In [49]:
merged = dfg.merge(full_df, on='ntaname' )
for bin in bins:
    print(bin)
    geo_layer = merged[(merged['bin'] == bin)] # geo
    try:
        geo_layer.to_file("data/neighborhoods/{}.geojson".format(bin), driver='GeoJSON')
    except:
        print('err')

0-500
501-1000
1001-1500
1501-2000
2001-2500
2501-3000
3001-3500
3501-4000
4001-4500
4501-5000


In [51]:

mergedB = dfgB.merge(full_dfB, on='borough' )
for bin in binsB:
    print(bin)
    geo_layer = mergedB[(mergedB['bin'] == bin)] # geo
    try:
        geo_layer.to_file("data/boroughs/{}.geojson".format(bin), driver='GeoJSON')
    except:
        print('err')

2000-2300
2301-2600
2601-2900
2901-3200
err
3201-3500


In [52]:
merged.columns

Index(['ntacode', 'shape_area', 'county_fips', 'ntaname', 'shape_leng',
       'boro_name', 'boro_code', 'geometry', 'Unnamed: 0', 'borough', '#trees',
       'area', 'health', 'trees/sq.mile', '#properties',
       'avg.landprice_thous$/acre', 'avg.propvalue_thous$/acre',
       'properties/sq.mile', 'centerLong', 'centerLat', 'hover', 'bin'],
      dtype='object')

In [53]:
merged.drop(columns={'shape_area', 'county_fips','boro_name','shape_leng', 'boro_code','Unnamed: 0'}, inplace = True)

In [54]:
merged.head()

,ntacode,ntaname,geometry,borough,#trees,area,health,trees/sq.mile,#properties,avg.landprice_thous$/acre,avg.propvalue_thous$/acre,properties/sq.mile,centerLong,centerLat,hover,bin
0,bk88,borough park,"MULTIPOLYGON (((-73.97605 40.63128, -73.97717 ...",brooklyn,5709.0,1.94,14.19,2947.09,13671.0,27.18,832.90,7057.23,-73.988661,40.630950,borough park<br>brooklyn<br>trees/sq.mile: 294...,2501-3000
1,qn51,murray hill,"MULTIPOLYGON (((-73.80379 40.77561, -73.80099 ...",queens,6838.0,1.88,14.36,3631.91,8719.0,32.13,776.19,4630.97,-73.809546,40.768352,murray hill<br>queens<br>trees/sq.mile: 3631.9...,3501-4000
2,qn27,east elmhurst,"MULTIPOLYGON (((-73.86110 40.76366, -73.85993 ...",queens,2055.0,0.71,14.06,2904.17,3385.0,22.56,674.05,4783.75,-73.868396,40.763352,east elmhurst<br>queens<br>trees/sq.mile: 2904...,2501-3000
3,qn07,hollis,"MULTIPOLYGON (((-73.75726 40.71814, -73.75589 ...",queens,2435.0,0.82,14.34,2965.95,3641.0,18.52,472.06,4434.91,-73.761137,40.710639,hollis<br>queens<br>trees/sq.mile: 2965.95<br>...,2501-3000
4,mn06,manhattanville,"MULTIPOLYGON (((-73.94608 40.82126, -73.94640 ...",manhattan,862.0,0.38,13.60,2257.07,473.0,315.56,4796.59,1238.51,-73.953782,40.817976,manhattanville<br>manhattan<br>trees/sq.mile: ...,2001-2500


In [55]:
mergedB.columns

Index(['boro_code', 'borough', 'shape_area', 'shape_leng', 'geometry',
       'Unnamed: 0', 'health', 'trees/sq.mile', 'avg.landprice_thous$/acre',
       'properties/sq.mile', 'centerLong', 'centerLat', 'hover', 'bin'],
      dtype='object')

In [56]:
mergedB.drop(columns={'boro_code', 'shape_area','shape_leng','Unnamed: 0'}, inplace = True)

In [23]:
type(merged), type(dfg)

(geopandas.geodataframe.GeoDataFrame, geopandas.geodataframe.GeoDataFrame)

In [24]:
for bin in bins:
    geo_layer = merged[(merged['bin'] == bin)] 
    geo_layer.to_file("data/{}.geojson".format(bin), driver='GeoJSON')

In [25]:
data = graph_objs.Data([
    graph_objs.Scattermapbox(
        lat=['40.7342'],
        lon=['-73.91251'],
        mode='markers',        
    )
])

layout = dict(
    height=600,
    autosize=True,
    hovermode='closest',
    margin=dict(t=0,r=0,l=0,b=0),
    mapbox=dict(
        layers=[
            dict(
                sourcetype = 'geojson',
                source = 'data/NeighborhoodTabulationAreas.json',
                type = 'fill',
                color = 'rgba(163,22,19,0.8)'
            )
        ],
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(
            lat=27.8,
            lon=-83
        ),
        pitch=0,
        zoom=5.2,
        style='light'        
    )
)    

fig = dict(data=data, layout=layout)

C:\Users\mskac\Anaconda3\lib\site-packages\plotly\graph_objs\_deprecations.py:40: DeprecationWarning:

plotly.graph_objs.Data is deprecated.
Please replace it with a list or tuple of instances of the following types
  - plotly.graph_objs.Scatter
  - plotly.graph_objs.Bar
  - plotly.graph_objs.Area
  - plotly.graph_objs.Histogram
  - etc.




In [26]:
py.plot(fig, filename='county-level-choropleths-python.html')


'county-level-choropleths-python.html'